# Analysis Notebook

This notebook documents the main analysis for the [ShotSpotter Routinely Missed Reported Shootings, City Data Shows](https://southsideweekly.com/shotspotter-routinely-missed-reported-shootings-city-data-shows/) story.

## Setup

In [1]:
import datetime as dt

import geopandas as gpd
import pandas as pd

from shotspotter import settings

We load the matched shooting/alert dataset from the public data directory:

In [2]:
matching_shootings_alerts = pd.read_csv(
    settings.DATA_DIR_PUBLIC / "matched_shootings_alerts_2023_2024.csv",
    parse_dates=["date_time", "date_time_alert"],
    index_col="id",
)
matching_shootings_alerts.head()

,case_number,date_time,latitude,longitude,type,place_description,police_district,location,location_in_meters,search_area,...,date,date_time_alert,distance_to_alert_in_meters,id_alert,latitude_alert,location_alert,location_in_meters_alert,longitude_alert,type_alert,detected
id,,,,,,,,,,,,,,,,,,,,,
SHOOT-JG134757-#1,JG134757,2023-01-30 13:13:00,41.749632,-87.664005,BATTERY,STREET,6,POINT (-87.664005346 41.749631654),POINT (-9758712.436015531 5123548.948779396),POLYGON ((-87.65677687240327 41.74963165399999...,...,2023-01-30,2023-01-30 13:12:52,126.383402,SST-387399,41.750466,POINT (-87.66381276 41.750466393),POINT (-9758690.99744008 5123673.500582644),-87.663813,MULTIPLE GUNSHOTS,True
SHOOT-JG446904-#2,JG446904,2023-10-01 22:44:00,41.827692,-87.680404,BATTERY,STREET,9,POINT (-87.680403976 41.827691918),POINT (-9760537.923156839 5135203.37299404),"POLYGON ((-87.67317550240327 41.827691918, -87...",...,2023-10-01,2023-10-01 22:44:19,120.646960,SST-35000300487,41.828074,POINT (-87.681358808 41.828073983),POINT (-9760644.214568872 5135260.450351795),-87.681359,MULTIPLE GUNSHOTS,True
HOM-JH175123-#1,JH175123,2024-03-04 13:19:00,41.844202,-87.705945,HOMICIDE,STREET,10,POINT (-87.705944654 41.844201654),POINT (-9763381.098426316 5137670.102050191),"POLYGON ((-87.6987161804033 41.84420165400001,...",...,2024-03-04,2024-03-04 13:18:36,25.859151,SST-79100114753,41.844374,POINT (-87.705962664 41.844374185),POINT (-9763383.103290344 5137695.883365209),-87.705963,MULTIPLE GUNSHOTS,True
HOM-JH175867-#1,JH175867,2024-03-04 23:50:00,41.887062,-87.755605,HOMICIDE,STREET,15,POINT (-87.755605346 41.887061654),POINT (-9768909.301372197 5144076.8117983835),POLYGON ((-87.74837687240328 41.88706165399999...,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
HOM-JG484795-#1,JG484795,2023-10-30 17:04:00,41.736540,-87.577130,HOMICIDE,PARKING LOT,4,POINT (-87.57713 41.7365405),POINT (-9749041.516736323 5121595.823287509),"POLYGON ((-87.56990152640327 41.7365405, -87.5...",...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


## Main Findings

### Overall Detection Rate

Calculating this is quite simple—we just find the percentage of shootings matched to alerts:

In [3]:
(
    matching_shootings_alerts["detected"]
    .loc[matching_shootings_alerts["date_time"] < dt.datetime(2024, 9, 1)]
    .value_counts(normalize=True)
    .map(lambda x: f"{x:.1%}")
)

detected
True     78.3%
False    21.7%
Name: proportion, dtype: object

### Detection Rate by Month

The same calculation, but broken down by month.

In [4]:
(
    matching_shootings_alerts
    .loc[matching_shootings_alerts["date_time"] < dt.datetime(2024, 9, 1)]
    .assign(month=matching_shootings_alerts["date_time"].map(lambda x: dt.datetime(x.year, x.month, 1)))
    .groupby("month")["detected"]
    .value_counts(normalize=True)
    .xs(True, level="detected")
    .map(lambda x: f"{x:.1%}")
)

month
2023-01-01    82.5%
2023-02-01    85.9%
2023-03-01    81.9%
2023-04-01    78.7%
2023-05-01    74.7%
2023-06-01    75.6%
2023-07-01    70.3%
2023-08-01    80.6%
2023-09-01    77.1%
2023-10-01    82.4%
2023-11-01    84.3%
2023-12-01    78.7%
2024-01-01    79.3%
2024-02-01    79.1%
2024-03-01    77.7%
2024-04-01    82.6%
2024-05-01    80.8%
2024-06-01    76.1%
2024-07-01    72.2%
2024-08-01    74.9%
Name: proportion, dtype: object

### Missed Shootings by Type

In the story, we write:
>That includes TK gun homicides, as well as TK nonfatal shootings and TK reckless firearm discharges that ShotSpotter apparently failed to alert police to.

We calculate these figures as follows:

In [5]:
(
    matching_shootings_alerts
    .loc[matching_shootings_alerts["date_time"] < dt.datetime(2024, 9, 1)]
    .groupby("type")["detected"]
    .value_counts()
    .xs(False, level="detected")
)

type
BATTERY                       627
HOMICIDE                      180
RECKLESS FIREARM DISCHARGE    415
ROBBERY                        48
Name: count, dtype: int64

### Missed Shootings by Type and Year

In the story, we write:
> In all of 2023, TK fatal and TK nonfatal shootings in the coverage area had no ShotSpotter alert. Through August of this year, ShotSpotter has missed TK fatal and TK nonfatal shootings in its coverage area.

We calculate these figures as follows:

In [6]:
(
    matching_shootings_alerts
    .loc[matching_shootings_alerts["date_time"] < dt.datetime(2024, 9, 1)]
    .assign(year=matching_shootings_alerts["date_time"].dt.year)
    .groupby(["year", "type"])["detected"]
    .value_counts()
    .xs(False, level="detected")
)

year  type                      
2023  BATTERY                       373
      HOMICIDE                      113
      RECKLESS FIREARM DISCHARGE    256
      ROBBERY                        24
2024  BATTERY                       254
      HOMICIDE                       67
      RECKLESS FIREARM DISCHARGE    159
      ROBBERY                        24
Name: count, dtype: int64